# Assignment 7

Dominic Heye Z1841746

# Part 1

In [13]:
import requests
import zipfile
import io

In [14]:
r1 = requests.get('http://faculty.cs.niu.edu/~dakoop/cs503-2021fa/a7/fg.zip')
r2 = requests.get('http://faculty.cs.niu.edu/~dakoop/cs503-2021fa/a7/hij.zip')
r3 = requests.get('http://faculty.cs.niu.edu/~dakoop/cs503-2021fa/a7/tu.zip')

In [15]:
r = [r1, r2, r3]
for i in r:
    z = zipfile.ZipFile(io.BytesIO(i.content))
    z.extractall()

# Part 2

In [16]:
import os
path_list = []

In [17]:
with os.scandir('data') as entries:
    for entry in entries:
        if entry.is_dir():
            directory = 'data/' + entry.name
            with os.scandir(directory) as sub_entries:
                for sub_entry in sub_entries:
                    path = directory + '/' + sub_entry.name
                    path_list.append(path)
        else:
            path = 'data/' + entry.name
            path_list.append(path)

# Part 3

In [18]:
import pandas as pd
from pandas import Series, DataFrame
from concurrent import futures
import concurrent.futures
final_df = pd.DataFrame()

In [19]:
def process(file_location):
    df = pd.read_csv(file_location, na_values=-999.0)
    df = df.query('data_class == "electricity"')
    return df

In [20]:
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_to_process = {executor.submit(process,file_location): file_location for file_location in path_list}
    data_list= []
    for future in concurrent.futures.as_completed(future_to_process):
        file_location = future_to_process[future]
        try:
            data = future.result()
            data_list.append(data)
            #print(data)
        except Exception as exc:
            print('%r generated an exception: %s' % (file_location, exc))\
            
#print(data_list)

final_df = pd.concat(data_list)
final_df

,county_name,com_name,year,month,data_class,data_field_display_name,unit,value,number_of_accounts
0,Fulton,Northville,2020,10,electricity,Total Consumption (T),MWh,520.628,718.0
1,Fulton,Northville,2020,10,electricity,Business Consumption (SC+O),MWh,157.420,87.0
2,Fulton,Northville,2020,10,electricity,Small Commercial Consumption (SC),MWh,NaN,81.0
3,Fulton,Northville,2020,10,electricity,Small Consumer Consumption (R+SC),MWh,NaN,712.0
4,Fulton,Northville,2020,10,electricity,Residential Consumption (R),MWh,363.208,631.0
...,...,...,...,...,...,...,...,...,...
10083,Jefferson,Adams,2019,10,electricity,Business Consumption (SC+O),MWh,NaN,72.0
10084,Jefferson,Adams,2019,10,electricity,Small Consumer Consumption (R+SC),MWh,438.691,675.0
10085,Jefferson,Adams,2019,10,electricity,All Other Consumption (O),MWh,NaN,2.0
10086,Jefferson,Adams,2019,10,electricity,Small Commercial Consumption (SC),MWh,148.403,70.0


In [21]:
df_2019 = final_df[final_df['year'] == 2019]
df_2020 = final_df[final_df['year'] == 2020]

In [22]:
df_2019.to_csv('2019.csv.gz')
df_2020.to_csv('2020.csv.gz')

In [23]:
df2019 = pd.read_csv('2019.csv.gz')
df2019.groupby('county_name').size()

county_name
Franklin     1134
Fulton        630
Genesee       798
Greene        840
Hamilton      504
Herkimer     1302
Jefferson    1680
Tioga         588
Tompkins      630
Ulster       1050
dtype: int64

In [24]:
df2020 = pd.read_csv('2020.csv.gz')
df2020.groupby('county_name').size()

county_name
Franklin     3724
Fulton       2100
Genesee      2660
Greene       2884
Hamilton     1680
Herkimer     4340
Jefferson    5600
Tioga        1960
Tompkins     2100
Ulster       3500
dtype: int64